## Exporting yolov5 model to Pytorch Mobile

WARNING: the following export was done using the [ultralytics yolov5 code](https://github.com/ultralytics/yolov5/blob/master/export.py), and the necessary adaptations were made so it would work in the Pytorch official [iOS demo app](https://github.com/pytorch/ios-demo-app/tree/master/ObjectDetection).

In this script we're more interested in the smallest model, the yolov5n, but it should work fine for larger ones.


### 0.5 Install requirements

In [7]:
%pip install -r requirements.txt

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached charset_normalizer-3.3.2-cp310-cp310-macosx_11_0_arm64.whl (120 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 1. Clone the yolov5 repo to export models
I'm cloning from the main, but for future reproduction you can reset to the commit that I used:
```
git reset --hard 100a423b66fee81e0a2915d5da934d7872f12c8c
```

In [1]:
YOLOV5_DIR = "third_party/yolov5"
!git -C {YOLOV5_DIR} pull || git clone https://github.com/ultralytics/yolov5.git {YOLOV5_DIR}

fatal: cannot change to 'third_party/yolov5': No such file or directory
Cloning into 'third_party/yolov5'...
remote: Enumerating objects: 16807, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 16807 (delta 187), reused 179 (delta 94), pack-reused 16459
Receiving objects: 100% (16807/16807), 15.56 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (11486/11486), done.


### 2. Export the yolov5n model to torchscript mobile compatible

Do not forget to include `--optimize` in the options, otherwise it would not convert for a mobile compatible model (torch-lite).

In [12]:
import os

MODEL_NAME = "yolov5n.pt"
if not os.path.exists(MODEL_NAME):
    !wget https://github.com/ultralytics/yolov5/releases/download/v7.0/{MODEL_NAME} -P .
!python3 third_party/yolov5/export.py --weights {MODEL_NAME} --include torchscript --optimize

export: data=third_party/yolov5/data/coco128.yaml, weights=['yolov5n.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=True, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript']
YOLOv5 🚀 v7.0-345-g8003649c Python-3.10.14 torch-2.3.1 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs

PyTorch: starting from yolov5n.pt with output shape (1, 25200, 85) (3.9 MB)

TorchScript: starting export with torch 2.3.1...
TorchScript: export success ✅ 0.9s, saved as yolov5n.torchscript (7.4 MB)

Export complete (1.2s)
Results saved to /Users/gustavofuhr/projects/object_detection_ios_comprehensive/yolov5_yolov8_ultralytics_to_pytorch_mobile
Detect:          python detect.py --weights yolov5n.torchscript 
Validate:        python val.py --weights

If the above worked as expected, you should have in the current dir the original Pytorch yolov5 model `yolov5n.pt` and the converted one `yolov5n.torchscript`. In the next step you will need to put the model in the sample xcode project.

### 3. Setup the iOS demo app for yolov5

Clone if doesn't exists

In [3]:
DEMO_DIR = "third_party/pytorch-ios-demo-app"
!git -C {DEMO_DIR} pull || git clone https://github.com/pytorch/ios-demo-app {TARGET_DIR}

Already up to date.


#### Install pods 
(IMPORTANT!) The version of the Lib-Torch specified in the Podfile will not work in the current version of the model/exporter. So we need to patch it.

In [4]:
import os

with open(f"{DEMO_DIR}/ObjectDetection/Podfile", "r") as f, \
        open(f"{DEMO_DIR}/ObjectDetection/Podfile_fixed_for_yolov5", "w") as f_out:
    content = f.readlines()
    for ln in content:
        if "LibTorch-Lite" in ln:
            ln = "  pod 'LibTorch-Lite', '~>1.12'\n"
        f_out.write(ln)

os.rename(f'{DEMO_DIR}/ObjectDetection/Podfile_fixed_for_yolov5', f'{DEMO_DIR}/ObjectDetection/Podfile')
print("Podfile updated")

Podfile updated


In [40]:
!cd {TARGET_DIR}/ObjectDetection && pod install

Analyzing dependencies
Generating Pods project
Integrating client project
Pod installation complete! There is 1 dependency from the Podfile and 1 total pod installed.

[!] Automatically assigning platform `iOS` with version `14.0` on target `ObjectDetection` because no platform was specified. Please specify a platform for this target in your Podfile. See `https://guides.cocoapods.org/syntax/podfile.html#platform`.


🚨 Copy the `yolo5n.torchscript` to the xcode project, modify the `Inference/ObjectDetector.swift` to reference this new file by changing the `let filePath` line to `if let filePath = Bundle.main.path(forResource: "yolov5n", ofType: "torchscript"),`. It should work 🤞
